In [1]:
import syft
import syft as sy
import numpy as np
from syft.core import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import random
from syft.core.frameworks.torch import utils as torch_utils
from torch.autograd import Variable
hook = sy.TorchHook(verbose=False)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice]

bob.add_workers([alice])
alice.add_workers([bob])


import msgpack
%load_ext line_profiler

In [13]:
x = sy.Variable(sy.FloatTensor([1,1,3])).send(bob)
%lprun \
-f sy._PointerTensor.handle_call \
-f torch_utils.wrap_command \
y = x + x

In [3]:
x = sy.Variable(sy.FloatTensor([1,1,3]))
%lprun \
-f torch_utils.prepare_child_command \
-f sy.core.workers.BaseWorker._execute_call \
-f sy._LocalTensor.handle_call \
y = x + x

In [4]:
%lprun -f sy.FloatTensor.__init__ x = sy.FloatTensor([5,3])

In [5]:
sy._LocalTensor.__init__

<function syft.core.frameworks.torch.tensor._LocalTensor.__init__>

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()

total_norm = 0
norm_type = 2
for p in model.parameters():
    param_norm = p.data.norm(norm_type)
    total_norm += param_norm ** norm_type
    
print(total_norm)

#worker = bob

#model.send(worker)
#for param in model.parameters():
#    print("Param Id: " + str(param.id_at_location))
#    print(worker._objects.keys())
#    assert(param.id_at_location in worker._objects.keys())
#pred = model(data)

12.415705714759735


In [7]:
torch.syft_tensor_codes._LocalTensor == 1

AttributeError: 'set' object has no attribute '_LocalTensor'

In [ ]:
%%time
for i in range(10000):
    torch_type = sy.FloatTensor([i,2])
    _ = torch_type.__class__

In [ ]:
def quick_test(obj):
    try:
        _ = torch.var_codes[obj]
        return True
    except KeyError:
        return False

In [ ]:
%%time
for i in range(10000):
    torch_type = sy.FloatTensor([i,2])
    _ = type(torch_type)

In [ ]:
print(torch.torch_exclude)

ws = [str(w) for w in list(range(1000))]
d = {}
for w in ws:
    d[w] = 'native_'+w


In [ ]:
%%time
for i in range(1000):
    loc = []
    for w in ws:
        loc.append(w)

In [ ]:
%%time
for i in range(1000):
    loc1 = []
    for w in ws:
        loc1.append('native_{}'.format(w))

In [ ]:
%%time
for i in range(1000):
    loc = []
    for w in ws:
        loc.append('native_'+w)

In [ ]:
%%time
for i in range(1000):
    loc = []
    for w in ws:
        loc.append(d[w])

In [ ]:
torch.syft_tensor_codes['_LocalTensor']

In [ ]:
torch_utils.type_code('__worker__').name

In [ ]:
getattr(torch.encoded_types, '__worker__')

In [ ]:
torch.types[1]

In [ ]:
from enum import IntEnum, auto

In [ ]:
encoded_types = IntEnum(
    'DynamicEnum',
    ['worker', 'foo']
)

In [ ]:
types = IntEnum('DynamicEnum', {'foo':42, 'bar':24})

In [ ]:
encoded_types.foo == 2

In [ ]:
import torch

In [ ]:
torch.LongTensor.old_add = torch.LongTensor.__add__